In [1]:
import furniture_bench  # noqa: F401

from src.behavior.base import Actor
from src.eval.load_model import load_bc_actor

import wandb
import numpy as np
import torch

from omegaconf import OmegaConf

from furniture_bench.envs.observation import DEFAULT_STATE_OBS
import hydra
from src.gym import turn_off_april_tags
from src.gym.env_rl_wrapper import ResidualPolicyEnvWrapper
from src.gym.furniture_sim_env import FurnitureRLSimEnv
from src.models.residual import ResidualPolicy


turn_off_april_tags()

api = wandb.Api()

device = "cuda" if torch.cuda.is_available() else "cpu"

Importing module 'gym_38' (/data/scratch/ankile/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data/scratch/ankile/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.0+cu121
Device count 1
/data/scratch/ankile/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data/scratch/ankile/.cache as PyTorch extensions root...
Emitting ninja build file /data/scratch/ankile/.cache/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


In [2]:
residual_run = api.run("residual-ppo-2/runs/sdoolikr")

cfg = OmegaConf.create(
    {
        **residual_run.config,
        "env": {"randomness": "low"},
        # "base_bc_poliy": "ankile/one_leg-diffusion-state-1/runs/7623y5vn",
    }
)

cfg

{'seed': 3482095094, 'debug': False, 'gamma': 0.999, 'wandb': {'entity': 'ankile', 'project': 'residual-ppo-2'}, 'vf_coef': 0.5, 'ent_coef': 0, 'headless': True, 'norm_adv': True, 'num_envs': 1024, 'clip_coef': 0.2, 'target_kl': 0.03, 'batch_size': 870400, 'clip_vloss': True, 'gae_lambda': 0.95, 'norm_grads': False, 'action_type': 'pos', 'act_rot_repr': 'rot_6d', 'lr_scheduler': {'name': 'cosine', 'warmup_steps': 5}, 'base_bc_poliy': 'ankile/one_leg-diffusion-state-1/runs/7623y5vn', 'eval_interval': 5, 'max_grad_norm': 0.5, 'num_env_steps': 850, 'update_epochs': 10, 'minibatch_size': 870400, 'num_iterations': 574, 'num_minibatches': 1, 'residual_policy': {'_target_': 'src.models.residual.ResidualPolicy', 'init_logstd': -1, 'action_scale': 0.05, 'action_head_std': 0, 'actor_activation': 'ReLU', 'actor_num_layers': 2, 'actor_hidden_size': 512, 'critic_activation': 'ReLU', 'critic_num_layers': 2, 'critic_hidden_size': 512, 'critic_last_layer_bias_const': 0.5}, 'total_timesteps': 500000000

In [3]:
env: FurnitureRLSimEnv = FurnitureRLSimEnv(
    act_rot_repr=cfg.act_rot_repr,
    action_type=cfg.action_type,
    april_tags=False,
    concat_robot_state=True,
    ctrl_mode="diffik",
    obs_keys=DEFAULT_STATE_OBS,
    furniture="one_leg",
    gpu_id=0,
    headless=True,  # cfg.headless,
    num_envs=128,  # cfg.num_envs,
    observation_space="state",
    randomness=cfg.env.randomness,
    max_env_steps=100_000_000,
)

# Load the behavior cloning actor
bc_actor: Actor = load_bc_actor(cfg.base_bc_poliy)

env: ResidualPolicyEnvWrapper = ResidualPolicyEnvWrapper(
    env,
    max_env_steps=cfg.num_env_steps,
    reset_on_success=cfg.reset_on_success,
    reset_on_failure=cfg.reset_on_failure,
)
env.set_normalizer(bc_actor.normalizer)

# Residual policy setup
residual_policy: ResidualPolicy = hydra.utils.instantiate(
    cfg.residual_policy,
    obs_shape=env.observation_space.shape,
    action_shape=env.action_space.shape,
)

residual_policy.to(device)

Making DiffIK controller with pos_scalar: 1.0, rot_scalar: 1.0
Stiffness: 1000.0, Damping: 200.0
Observation keys: ['robot_state/ee_pos', 'robot_state/ee_quat', 'robot_state/ee_pos_vel', 'robot_state/ee_ori_vel', 'robot_state/gripper_width', 'parts_poses']
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Using SDF cache directory '/afs/csail.mit.edu/u/a/ankile/.isaacgym/sdf_V100'
~!~!~! Loaded/Cooked SDF triangle mesh 0 @ 0xcc64e2d0, resolution=512, spacing=0.000317
  ~!~! Bounds:  (-0.081250, 0.081250) (-0.015685, 0.015565) (-0.081250, 0.081251)
  ~!~! Extents: (0.162500, 0.031250, 0.162501)
  ~!~! Resolution: (512, 99, 512)
~!~!~! Loaded/Cooked SDF triangle mesh 1 @ 0xcc4a32b0, resolution=512, spacing=0.000171
  ~!~! Bounds:  (-0.015000, 0.015000) (-0.056250, 0.031250) (-0.014383, 0.015618)
  ~!~! Extents: (0.030000, 0.087500, 0.030001)
  ~!~! Resolution: (176, 512, 176)
~!~!~! Loaded/Cooked SDF triangle mesh 2 @ 0xcddf22d0, 

/data/scratch/ankile/miniconda3/envs/rr/lib/python3.8/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Total parameters: 66.69M
normalizer: 0.00M parameters
loss_fn: 0.00M parameters
model: 66.69M parameters
./models/star-destroyer-10/actor_chkpt_best_success_rate.pt
Total parameters: 0.59M
actor_mean: 0.30M parameters
critic: 0.29M parameters


ResidualPolicy(
  (actor_mean): Sequential(
    (0): Linear(in_features=61, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=False)
  )
  (critic): Sequential(
    (0): Linear(in_features=61, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [4]:
from tqdm import trange


next_obs = env.reset()
bc_actor.reset()

total_reward = 0


for step in trange(0, 850):

    # Get the base normalized action
    base_naction = bc_actor.action_normalized(next_obs)

    # Process the obs for the residual policy
    next_obs = env.process_obs(next_obs)
    next_residual_obs = torch.cat([next_obs, base_naction], dim=-1)

    with torch.no_grad():
        residual_naction_samp, logprob, _, value, naction_mean = (
            residual_policy.get_action_and_value(next_residual_obs)
        )

    residual_naction = naction_mean
    naction = base_naction + residual_naction * cfg.residual_policy.action_scale

    next_obs, reward, next_done, truncated, infos = env.step(naction)

    total_reward += reward.sum()


# Calculate the success rate
total_reward / env.num_envs

  0%|          | 0/850 [00:00<?, ?it/s]/data/scratch/ankile/miniconda3/envs/rr/lib/python3.8/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
 65%|██████▌   | 555/850 [01:25<00:52,  5.65it/s]INFO: square_table_top (id: 0), square_table_leg4 (id: 4) are assembled.
INFO: Assembled Set [square_table_top (id: 0), square_table_leg4 (id: 4)]
 66%|██████▋   | 565/850 [01:27<00:47,  6.01it/s]INFO: square_table_top (id: 0), square_table_leg4 (id: 4) are assembled.
INFO: Assembled Set [square_table_top (id: 0), square_table_leg4 (id: 4)]
 67%|██████▋   | 572/850 [01:28<00:48,  5.79it/s]INFO: square_table_top (id: 0), square_table_leg4 (id: 4) are assembled.
INFO: Assembled Set [square_table_top (id: 0), square_table_le

tensor(0.3359, device='cuda:0')

In [5]:
# Only base policy: 28%
# With residual 34%...?